<a href="https://colab.research.google.com/github/VishnuReddy25/DL/blob/main/BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from transformers import TFAutoModel, AutoTokenizer
from sklearn.model_selection import train_test_split
import numpy as np

# Step 1: Prepare Dataset
texts = [
    "I love this movie!",
    "This is terrible.",
    "I feel amazing today.",
    "Worst experience ever.",
    "Best product I bought!",
    "I hate this."
]
labels = [1, 0, 1, 0, 1, 0]  # 1=Positive, 0=Negative

# Split data
train_texts, val_texts, train_labels, val_labels = train_test_split(
    texts, labels, test_size=0.2, random_state=42
)

# Step 2: Initialize Tokenizer
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Tokenize texts
train_encodings = tokenizer(train_texts, truncation=True, padding=True, return_tensors="tf")
val_encodings = tokenizer(val_texts, truncation=True, padding=True, return_tensors="tf")

# Step 3: Build Model
bert_base = TFAutoModel.from_pretrained(model_name)

# Input layers
input_ids = tf.keras.Input(shape=(None,), dtype=tf.int32, name="input_ids")
attention_mask = tf.keras.Input(shape=(None,), dtype=tf.int32, name="attention_mask")

# BERT outputs
outputs = bert_base(input_ids, attention_mask=attention_mask)

# Classification head
cls_output = outputs.last_hidden_state[:, 0, :]  # [CLS] token
x = tf.keras.layers.Dense(128, activation='relu')(cls_output)
x = tf.keras.layers.Dropout(0.2)(x)
final_output = tf.keras.layers.Dense(1, activation='sigmoid')(x)

model = tf.keras.Model(inputs=[input_ids, attention_mask], outputs=final_output)

# Step 4: Compile and Train
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=3e-5),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

history = model.fit(
    x={'input_ids': train_encodings['input_ids'],
       'attention_mask': train_encodings['attention_mask']},
    y=np.array(train_labels),
    validation_data=(
        {'input_ids': val_encodings['input_ids'],
         'attention_mask': val_encodings['attention_mask']},
        np.array(val_labels)
    ),
    batch_size=2,
    epochs=3
)

# Step 5: Prediction Function
def predict_sentiment(text):
    tokens = tokenizer(text, return_tensors="tf", truncation=True, padding=True)
    prediction = model.predict({'input_ids': tokens['input_ids'],
                               'attention_mask': tokens['attention_mask']})
    return "Positive" if prediction[0][0] > 0.5 else "Negative"

# Test predictions
print(predict_sentiment("I absolutely loved it!"))  # Should output: Positive
print(predict_sentiment("This is the worst thing ever."))  # Should output: Negative

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

Epoch 1/3


2/2 [==============================] - 45s 2s/step - loss: 0.7280 - accuracy: 0.5000 - val_loss: 0.6030 - val_accuracy: 0.5000
Epoch 2/3
2/2 [==============================] - 0s 93ms/step - loss: 0.5154 - accuracy: 0.7500 - val_loss: 0.5197 - val_accuracy: 1.0000
Epoch 3/3
1/1 [==============================] - 4s 4s/step
Positive
1/1 [==============================] - 3s 3s/step
Negative
